# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4bd4099be0>
├── 'a' --> tensor([[ 0.2811,  0.8114, -0.4701],
│                   [ 0.6287,  1.0713, -0.3039]])
└── 'x' --> <FastTreeValue 0x7f4bd4099a90>
    └── 'c' --> tensor([[ 0.3018, -0.3466,  0.6039,  2.0512],
                        [ 1.0164,  0.6363, -0.9404,  0.0115],
                        [ 0.5847, -0.9374, -0.5833, -0.2111]])

In [4]:
t.a

tensor([[ 0.2811,  0.8114, -0.4701],
        [ 0.6287,  1.0713, -0.3039]])

In [5]:
%timeit t.a

66.5 ns ± 0.0582 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4bd4099be0>
├── 'a' --> tensor([[ 0.5645, -0.5826, -0.2576],
│                   [ 1.9832, -1.0956, -2.4496]])
└── 'x' --> <FastTreeValue 0x7f4bd4099a90>
    └── 'c' --> tensor([[ 0.3018, -0.3466,  0.6039,  2.0512],
                        [ 1.0164,  0.6363, -0.9404,  0.0115],
                        [ 0.5847, -0.9374, -0.5833, -0.2111]])

In [7]:
%timeit t.a = new_value

71.4 ns ± 0.0609 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2811,  0.8114, -0.4701],
               [ 0.6287,  1.0713, -0.3039]]),
    x: Batch(
           c: tensor([[ 0.3018, -0.3466,  0.6039,  2.0512],
                      [ 1.0164,  0.6363, -0.9404,  0.0115],
                      [ 0.5847, -0.9374, -0.5833, -0.2111]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2811,  0.8114, -0.4701],
        [ 0.6287,  1.0713, -0.3039]])

In [11]:
%timeit b.a

59.3 ns ± 0.0236 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0893,  0.3898, -0.0540],
               [ 0.3514, -1.0280, -1.0211]]),
    x: Batch(
           c: tensor([[ 0.3018, -0.3466,  0.6039,  2.0512],
                      [ 1.0164,  0.6363, -0.9404,  0.0115],
                      [ 0.5847, -0.9374, -0.5833, -0.2111]]),
       ),
)

In [13]:
%timeit b.a = new_value

502 ns ± 0.0854 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

799 ns ± 0.0627 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

147 µs ± 509 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4bd4094820>
├── 'a' --> tensor([[[ 0.2811,  0.8114, -0.4701],
│                    [ 0.6287,  1.0713, -0.3039]],
│           
│                   [[ 0.2811,  0.8114, -0.4701],
│                    [ 0.6287,  1.0713, -0.3039]],
│           
│                   [[ 0.2811,  0.8114, -0.4701],
│                    [ 0.6287,  1.0713, -0.3039]],
│           
│                   [[ 0.2811,  0.8114, -0.4701],
│                    [ 0.6287,  1.0713, -0.3039]],
│           
│                   [[ 0.2811,  0.8114, -0.4701],
│                    [ 0.6287,  1.0713, -0.3039]],
│           
│                   [[ 0.2811,  0.8114, -0.4701],
│                    [ 0.6287,  1.0713, -0.3039]],
│           
│                   [[ 0.2811,  0.8114, -0.4701],
│                    [ 0.6287,  1.0713, -0.3039]],
│           
│                   [[ 0.2811,  0.8114, -0.4701],
│                    [ 0.6287,  1.0713, -0.3039]]])
└── 'x' --> <FastTreeValue 0x7f4b1d409bb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 68 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4bd40a14c0>
├── 'a' --> tensor([[ 0.2811,  0.8114, -0.4701],
│                   [ 0.6287,  1.0713, -0.3039],
│                   [ 0.2811,  0.8114, -0.4701],
│                   [ 0.6287,  1.0713, -0.3039],
│                   [ 0.2811,  0.8114, -0.4701],
│                   [ 0.6287,  1.0713, -0.3039],
│                   [ 0.2811,  0.8114, -0.4701],
│                   [ 0.6287,  1.0713, -0.3039],
│                   [ 0.2811,  0.8114, -0.4701],
│                   [ 0.6287,  1.0713, -0.3039],
│                   [ 0.2811,  0.8114, -0.4701],
│                   [ 0.6287,  1.0713, -0.3039],
│                   [ 0.2811,  0.8114, -0.4701],
│                   [ 0.6287,  1.0713, -0.3039],
│                   [ 0.2811,  0.8114, -0.4701],
│                   [ 0.6287,  1.0713, -0.3039]])
└── 'x' --> <FastTreeValue 0x7f4bd40a16a0>
    └── 'c' --> tensor([[ 0.3018, -0.3466,  0.6039,  2.0512],
                        [ 1.0164,  0.6363, -0.9404,  0.0115],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 33.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.8 µs ± 81.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.3018, -0.3466,  0.6039,  2.0512],
                       [ 1.0164,  0.6363, -0.9404,  0.0115],
                       [ 0.5847, -0.9374, -0.5833, -0.2111]],
              
                      [[ 0.3018, -0.3466,  0.6039,  2.0512],
                       [ 1.0164,  0.6363, -0.9404,  0.0115],
                       [ 0.5847, -0.9374, -0.5833, -0.2111]],
              
                      [[ 0.3018, -0.3466,  0.6039,  2.0512],
                       [ 1.0164,  0.6363, -0.9404,  0.0115],
                       [ 0.5847, -0.9374, -0.5833, -0.2111]],
              
                      [[ 0.3018, -0.3466,  0.6039,  2.0512],
                       [ 1.0164,  0.6363, -0.9404,  0.0115],
                       [ 0.5847, -0.9374, -0.5833, -0.2111]],
              
                      [[ 0.3018, -0.3466,  0.6039,  2.0512],
                       [ 1.0164,  0.6363, -0.9404,  0.0115],
                       [ 0.5847, -0.9374, -0.5833, -0.2111]],

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.3018, -0.3466,  0.6039,  2.0512],
                      [ 1.0164,  0.6363, -0.9404,  0.0115],
                      [ 0.5847, -0.9374, -0.5833, -0.2111],
                      [ 0.3018, -0.3466,  0.6039,  2.0512],
                      [ 1.0164,  0.6363, -0.9404,  0.0115],
                      [ 0.5847, -0.9374, -0.5833, -0.2111],
                      [ 0.3018, -0.3466,  0.6039,  2.0512],
                      [ 1.0164,  0.6363, -0.9404,  0.0115],
                      [ 0.5847, -0.9374, -0.5833, -0.2111],
                      [ 0.3018, -0.3466,  0.6039,  2.0512],
                      [ 1.0164,  0.6363, -0.9404,  0.0115],
                      [ 0.5847, -0.9374, -0.5833, -0.2111],
                      [ 0.3018, -0.3466,  0.6039,  2.0512],
                      [ 1.0164,  0.6363, -0.9404,  0.0115],
                      [ 0.5847, -0.9374, -0.5833, -0.2111],
                      [ 0.3018, -0.3466,  0.6039,  2.0512],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

338 µs ± 2.53 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
